In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import _pickle as pickle
from os import listdir
from os.path import join, isfile

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras import backend as K
K.set_session(session)

from imgaug import augmenters as iaa
from keras.preprocessing import image
from keras.utils import to_categorical, plot_model

from keras.applications import xception
from keras.applications.xception import preprocess_input

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score


Using TensorFlow backend.


In [2]:
def read_img(img_id, train_or_test, size=None):
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

def augment(img):
    images = np.zeros((NUM_AUGS, img.shape[0], img.shape[1], 3))
    images[0] = img
    
    # Randomly add noice and change color for following augmentations
    random_aug = iaa.OneOf([
        iaa.Multiply((0.9, 1.1), per_channel=1),
        iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.08*255), per_channel=0.5)
    ])
    
    # Flip horizontally
    images[1] = iaa.Fliplr(1).augment_images([img])[0]
    
    # Rotate both flipped images
    images[2] = iaa.Sequential([random_aug, iaa.Affine(rotate=(13, 16), mode='edge')]).augment_images([images[0]])[0]
    images[3] = iaa.Sequential([random_aug, iaa.Affine(rotate=(-13, -16), mode='edge')]).augment_images([images[0]])[0]
    images[4] = iaa.Sequential([random_aug, iaa.Affine(rotate=(13, 16), mode='edge')]).augment_images([images[1]])[0]
    images[5] = iaa.Sequential([random_aug, iaa.Affine(rotate=(-13, -16), mode='edge')]).augment_images([images[1]])[0]
    
    # Shear both flipped images
    images[6] = iaa.Sequential([random_aug, iaa.Affine(shear=(17, 22), mode='edge')]).augment_images([images[0]])[0]
    images[7] = iaa.Sequential([random_aug, iaa.Affine(shear=(-17, -22), mode='edge')]).augment_images([images[0]])[0]
    images[8] = iaa.Sequential([random_aug, iaa.Affine(shear=(17, 22), mode='edge')]).augment_images([images[1]])[0]
    images[9] = iaa.Sequential([random_aug, iaa.Affine(shear=(-17, -22), mode='edge')]).augment_images([images[1]])[0]
    
    return images

def add_augmented_labels(ls, inds):
    temp_augs = np.tile(np.arange(NUM_AUGS), (len(inds), 1))
    temp_inds = inds.values.reshape((len(inds), 1)) * NUM_AUGS
    new_inds = np.add(temp_inds, temp_augs).reshape(len(inds)*NUM_AUGS)
    new_ls = ls.reindex(np.repeat(ls.index.values, NUM_AUGS), method='ffill')
    new_ls = new_ls.iloc[new_inds, :]
    return new_ls, new_inds

In [3]:
# Load labels
NUM_CLASSES = 120
SEED = 1993
NUM_AUGS = 10
np.random.seed(seed=SEED)
data_dir = '../data'

labels = pd.read_csv(join(data_dir, 'labels.csv'))
print('Number of all train images: {}'.format(len(labels)))
print("Train data has {} classes.".format(len(labels.groupby('breed').count())))
assert len(labels.groupby('breed').count()) == NUM_CLASSES, 'Number of classes in training set is not 120!'

sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print('Number of all test images: {}'.format(len(sample_submission)))

# Split to train and validation sets
temp_labels_tr = labels.groupby('breed').apply(pd.DataFrame.sample, frac=0.8)
temp_labels_val = labels.loc[~labels['id'].isin(temp_labels_tr['id'])]
l_tr, inds_tr = add_augmented_labels(labels, temp_labels_tr.index.levels[1])
l_val, inds_val = add_augmented_labels(labels, temp_labels_val.index)

breed_index = {label:i for i,label in enumerate(np.unique(l_tr.breed))}
l_tr_temp = [breed_index[label] for label in l_tr.breed]
l_val_temp = [breed_index[label] for label in l_val.breed]
y_tr = to_categorical(l_tr_temp ,num_classes=120)
y_val = to_categorical(l_val_temp ,num_classes=120)

print('y_tr shape: {}'.format(y_tr.shape))
print('y_val shape: {}'.format(y_val.shape))

Number of all train images: 10222
Train data has 120 classes.
Number of all test images: 10357
y_tr shape: (81850, 120)
y_val shape: (20370, 120)


In [4]:
# Get bottelneck features
INPUT_SIZE = 299
POOLING = 'avg'
filename = data_dir + '//train//xs_bf_xception_aug'

if isfile(filename):
    # Load from file
    print('Loading from {}'.format(filename))
    with open(filename, 'rb') as fp:
        xs_bf = pickle.load(fp)
    print('xs_bf shape: {} size: {:,}'.format(xs_bf.shape, xs_bf.size))
else:
    # Preprocess images
    preprocess_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
    
    for i, img_id in enumerate(labels['id']):
        
        # Read image
        img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
        
        # Augment image
        images = augment(img)
        
        # Preprocess to model's liking (usually scales down to 0-1 range)
        images = map(lambda au_img: preprocess_input(np.expand_dims(au_img.copy(), axis=0)), images)
        images = np.asarray(list(images)).reshape((10, INPUT_SIZE, INPUT_SIZE, 3))
        
        # Retrieve bottleneck features
        x_bf_raw = preprocess_bottleneck.predict(images, batch_size=int(NUM_AUGS/2))
        
        # Store bottleneck features in xs_bf DataFrame
        x_bf = pd.DataFrame(x_bf_raw)
        x_bf['id'] = img_id
        if i == 0:
            xs_bf = x_bf
        else:
            xs_bf = pd.concat([xs_bf, x_bf], axis=0)
            
        if i % 100 == 0:
            print('Iteration {:5d} | Bottlenecks shape: {} size: {}'.format(i, xs_bf.shape, xs_bf.size))
    print('All train bottleneck features shape: {} size: {:,}'.format(xs_bf.shape, xs_bf.size))
    
    # Save into file
    print('Dumping into {}'.format(filename))
    with open(filename, 'wb') as fp:
        pickle.dump(xs_bf, fp)

# Split to train/val sets
x_tr = xs_bf.values[inds_tr, :-1]
print('Train bottleneck features shape: {} size: {:,}'.format(x_tr.shape, x_tr.size))

x_val = xs_bf.values[inds_val, :-1]
print('Validation bottleneck features shape: {} size: {:,}'.format(x_val.shape, x_val.size))

x_bf_raw = None
xs_bf = None

Loading from ../data//train//xs_bf_xception_aug
xs_bf shape: (102220, 2049) size: 209,448,780
Train bottleneck features shape: (81850, 2048) size: 167,628,800
Validation bottleneck features shape: (20370, 2048) size: 41,717,760


In [5]:
# Select original images for model comparison
x_tr_orig = x_tr[::10]
x_val_orig = x_val[::10]
y_tr_orig = y_tr[::10]
y_val_orig = y_val[::10]

In [6]:
# Logistic regression on all augmented images
logreg_aug = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg_aug.fit(x_tr, (y_tr * range(NUM_CLASSES)).sum(axis=1))

# Logistic regression on original images only
logreg_orig = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg_orig.fit(x_tr_orig, (y_tr_orig * range(NUM_CLASSES)).sum(axis=1))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=1993, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [85]:
loss = []
acc = []
models = [logreg_orig, logreg_aug]

for i in range(2):
    model = models[i]
    loss.append([])
    acc.append([])
    
    # validate on all images
    xv = x_val
    yv = y_val
    valid_probs = model.predict_proba(xv)
    loss[i].append(log_loss(yv, valid_probs))
    acc[i].append(accuracy_score((yv * range(NUM_CLASSES)).sum(axis=1), np.argmax(valid_probs, axis=1)))

    # validate on original images
    xv = x_val_orig
    yv = y_val_orig
    valid_probs = model.predict_proba(xv)
    loss[i].append(log_loss(yv, valid_probs))
    acc[i].append(accuracy_score((yv * range(NUM_CLASSES)).sum(axis=1), np.argmax(valid_probs, axis=1)))

    # validate on all images and average the predictions
    xv = x_val
    yv = y_val_orig
    valid_probs = model.predict_proba(xv)
    valid_probs = np.mean(valid_probs.reshape((int(valid_probs.shape[0]/10), 10, 120)), axis=1)
    valid_probs = valid_probs / np.sum(valid_probs, axis=0)
    loss[i].append(log_loss(yv, valid_probs))
    acc[i].append(accuracy_score((yv * range(NUM_CLASSES)).sum(axis=1), np.argmax(valid_probs, axis=1)))

    # validate on original and flipped images, then average predictions
    xv = np.concatenate((x_val[::10], x_val[1::10]))
    yv = y_val_orig
    valid_probs = model.predict_proba(xv)
    valid_probs = np.concatenate((valid_probs[:int(valid_probs.shape[0]/2)], valid_probs[int(valid_probs.shape[0]/2):]), axis=1)
    valid_probs = np.mean(valid_probs.reshape((int(valid_probs.shape[0]), 2, 120)), axis=1)
    valid_probs = valid_probs / np.sum(valid_probs, axis=0)
    loss[i].append(log_loss(yv, valid_probs))
    acc[i].append(accuracy_score((yv * range(NUM_CLASSES)).sum(axis=1), np.argmax(valid_probs, axis=1)))

In [86]:
notes = [
    'validate on all images',
    'validate on original images',
    'validate on all images and average the predictions',
    'validate on original and flipped images, then average predictions'
]
print('  Original  |  Augmented')
print(' loss  acc  |  loss  acc  ')
for i in range(len(loss[0])):
    print('{:.3f} {:.1%} | {:.3f} {:.1%} <- {}'.format(loss[0][i], acc[0][i], loss[1][i], acc[1][i], notes[i]))

  Original  |  Augmented
 loss  acc  |  loss  acc  
0.588 82.6% | 0.613 83.4% <- validate on all images
0.330 89.2% | 0.395 88.6% <- validate on original images
0.412 88.6% | 0.399 87.8% <- validate on all images and average the predictions
0.312 89.8% | 0.357 89.1% <- validate on original and flipped images, then average predictions


Up here it looks sad. Augmentation actually makes it worse.

My guess is that it actually increases overfitting because the augmented bottleneck features are too similar to the original. Further exploration in xception augmented notebook.

Note that the best performance is achieved by the original model when evaluated on 2 (original and flipped) images. This was also discovered in the xception augmented notebook.

In [ ]:
# Get test bottelneck features
INPUT_SIZE = 299
POOLING = 'avg'
filename = data_dir + '//test//xs_bf_xception_aug'

if isfile(filename):
    print('File {} already exists!'.format(filename))
else:
    # Preprocess images
    preprocess_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
    
    for i, img_id in enumerate(sample_submission['id']):
        
        # Read image
        img = read_img(img_id, 'test', (INPUT_SIZE, INPUT_SIZE))
        
        # Augment image - Not for test images!
        images = [img]
        
        # Preprocess to model's liking (usually scales down to 0-1 range)
        x = list(map(lambda au_img: preprocess_input(np.expand_dims(au_img.copy(), axis=0)), images))
        
        # Retrieve bottleneck features
        x_bf_raw = preprocess_bottleneck.predict(x, batch_size=1)
        
        # Store bottleneck features in xs_bf DataFrame
        x_bf = pd.DataFrame(x_bf_raw)
        x_bf['id'] = img_id
        if i == 0:
            xs_bf = x_bf
        else:
            xs_bf = pd.concat([xs_bf, x_bf], axis=0)
            
        if i % 100 == 0:
            print('Iteration {:5d} | Bottlenecks shape: {} size: {}'.format(i, xs_bf.shape, xs_bf.size))
    print('All test bottleneck features shape: {} size: {:,}'.format(xs_bf.shape, xs_bf.size))
    
    # Save into file
    print('Dumping into {}'.format(filename))
    with open(filename, 'wb') as fp:
        pickle.dump(xs_bf, fp)

x_bf_raw = None
xs_bf = None